In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import shap

In [10]:
data = pd.read_csv('./data/data_all.csv')
data.head()

,粗醇(t),精醇(t),二甲醚(t),醋酸(t),一厂乙二醇(t),二厂乙二醇(t),液体产品(t),乙腈(t),发电量(万度),一期有效气(万Nm3),...,二期原煤(t),下网电量(万kwh),一期用电量(万度),二期用电量(万度),预脱盐水(t),软化水(t),市政污水处理量(m3/d),新城提升泵供水(m3/d),一期酸脱年耗醇,二期酸脱年耗醇
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,23.0,6.3,17.2,4342.0,44718.0,1336.0,7700.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,25.0,6.4,18.1,2426.0,52730.0,1726.0,8801.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,24.0,6.1,17.7,1248.0,41558.0,2104.0,7970.0,0.0,0.0


In [11]:
new_columns = ['cuchun', 'jingchun', 'erjiami', 'cusuan', 'yichangyierchun', 'erchangyierchun', 'yetichanpin', 'yijing', 'fadianliang', 'yiqiyouxiaoqi', 'erqiyouxiaoqi', 'yierchunqingzufen', 'yierchunzhongzufen', 'ranmei', 'yiqiyuanmei', 'erqiyuanmei', 'xiawangdianliang', 'yiqiyongdianliang', 'erqiyongdianliang', 'yutuoyanshui', 'ranhuashui', 'shizhengwushuichuliliang', 'xinchengtishengbenggongshui', 'yiqisuantuonianhaochun', 'erqisuantuonianhaochun']

data.columns = new_columns
data = data.fillna(0)
data.head()

,cuchun,jingchun,erjiami,cusuan,yichangyierchun,erchangyierchun,yetichanpin,yijing,fadianliang,yiqiyouxiaoqi,...,erqiyuanmei,xiawangdianliang,yiqiyongdianliang,erqiyongdianliang,yutuoyanshui,ranhuashui,shizhengwushuichuliliang,xinchengtishengbenggongshui,yiqisuantuonianhaochun,erqisuantuonianhaochun
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,23.0,6.3,17.2,4342.0,44718.0,1336.0,7700.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,25.0,6.4,18.1,2426.0,52730.0,1726.0,8801.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,24.0,6.1,17.7,1248.0,41558.0,2104.0,7970.0,0.0,0.0


In [12]:
from sklearn.model_selection import train_test_split
np.random.seed(45)

X = data.iloc[:, 0:13]
Y = data.iloc[:, 13:25]


X.shape, Y.shape

((365, 13), (365, 12))

In [13]:
X_train, X_holdout, Y_train, Y_holdout = train_test_split(X, Y, test_size=0.1)

In [16]:
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()

# Fit your linear model
linear_model.fit(X_train, Y_train)

# Predict housing prices on the test set
Y_pred = linear_model.predict(X_holdout)

(12,)

In [17]:
def rmse(actual_y, predicted_y):
    return np.sqrt(np.mean((predicted_y - actual_y)**2))

train_error = rmse(Y_train, linear_model.predict(X_train))
holdout_error = rmse(Y_holdout, Y_pred)

print("Training RMSE:", train_error)
print("Holdout RMSE:", holdout_error)


Training RMSE: 2018.4647426214367
Holdout RMSE: 2132.3402686512622


## SHAP display

In [20]:

background = shap.utils.sample(X, 100)
explainer = shap.Explainer(linear_model.predict, background)
shap_values = explainer(X)

PermutationExplainer explainer: 366it [00:12,  8.48it/s]                         


In [33]:
shap_values

.values =
array([[[ 6.64194548e+02, -6.29007324e+01,  9.63487507e+00, ...,
         -5.12534897e+03, -2.39314112e+01,  1.56343439e+02],
        [-5.20162715e+01, -4.22976705e+01, -1.41839195e+01, ...,
         -1.09487793e+03, -2.45783857e+01, -7.76535448e+00],
        [-3.40469766e+01, -1.35413750e+01, -4.56725142e+01, ...,
         -2.25677923e+02,  1.12032458e+01,  2.15439626e+01],
        ...,
        [-7.42986450e+02,  8.85292943e+01, -1.53929294e+03, ...,
          5.23374085e+03,  1.42873842e+01, -1.60469726e+02],
        [ 6.15255702e+00, -3.89313831e+01, -4.32461228e+01, ...,
         -3.16429902e+02, -5.78852418e+01, -4.18830607e+01],
        [ 9.37072302e+00, -3.01992040e+01, -2.02439924e+01, ...,
         -3.58037485e+02, -5.81791272e+01, -5.52675042e+01]],

       [[ 6.64194548e+02, -6.29007324e+01,  9.63487507e+00, ...,
         -5.12534897e+03, -2.39314112e+01,  1.56343439e+02],
        [-5.20162715e+01, -4.22976705e+01, -1.41839195e+01, ...,
         -1.09487793e+03, -2

### Create the summary plot for a specific output/label/target.